In [2]:
import udapi
import collections
import pandas as pd

In [49]:
talb_doc = udapi.Document('/home/norrman/GitHub/UD_Swedish-Talbanken/not-to-release/output/temp/sv5.conllu')
pud_doc = udapi.Document('/home/norrman/GitHub/UD_Swedish-PUD/sv_pud-ud-test.conllu')
lines_train_doc = udapi.Document('/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-ud-train.conllu')
lines_test_doc = udapi.Document('/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-ud-test.conllu')
lines_dev_doc = udapi.Document('/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-ud-dev.conllu')
talb_doc[0].draw()
pud_doc[0].draw()
lines_train_doc[0].draw()
lines_test_doc[0].draw()
lines_dev_doc[0].draw()


# sent_id = sv-ud-train-1
# text = Individuell beskattning av arbetsinkomster
─┮
 │ ╭─╼ Individuell ADJ amod
 ╰─┾ beskattning NOUN root
   │ ╭─╼ av ADP case
   ╰─┶ arbetsinkomster NOUN nmod

# sent_id = n01001011
# text = ”Fast mycket av den digitala övergången är utan tidigare motstycke i USA, är det fredliga överlämnandet av makten inte det”, skrev Obamas specialassistent Kori Schulman i ett blogginlägg i måndags.
─┮
 │ ╭─╼ ” PUNCT punct
 │ │ ╭─╼ Fast SCONJ mark
 │ │ ┢─┮ mycket PRON nsubj
 │ │ │ │ ╭─╼ av ADP case
 │ │ │ │ ┢─╼ den DET det
 │ │ │ │ ┢─╼ digitala ADJ amod
 │ │ │ ╰─┶ övergången NOUN nmod
 │ │ ┢─╼ är AUX cop
 │ │ ┢─╼ utan ADP case
 │ │ ┢─╼ tidigare ADJ amod
 │ ┢─┾ motstycke NOUN advcl
 │ │ │ ╭─╼ i ADP case
 │ │ ┡─┶ USA PROPN obl
 │ │ ╰─╼ , PUNCT punct
 │ ┢─╼ är AUX cop
 │ │ ╭─╼ det DET det
 │ │ ┢─╼ fredliga ADJ amod
 │ ┢─┾ överlämnandet NOUN nsubj
 │ │ │ ╭─╼ av ADP case
 │ │ ╰─┶ makten NOUN nmod
 │ ┢─╼ inte PART advmod
 ╰─┾ det PRON root
   ┡─╼ ” PUNCT punct
   │ ╭─╼ , PUN

In [75]:
import inspect
inspect.getmembers(talb_doc[0])

[('__class__', udapi.core.bundle.Bundle),
 ('__delattr__',
  <method-wrapper '__delattr__' of Bundle object at 0x722ee9558b00>),
 ('__dir__', <function Bundle.__dir__()>),
 ('__doc__',
  'Bundle represents one sentence in an UD document.\n\n    A bundle contains one or more trees. More trees are needed e.g. in case of\n    parallel treebanks where each tree represents a translation of the sentence\n    in a different languages.\n    Trees in one bundle are distinguished by a zone label.\n    '),
 ('__eq__', <method-wrapper '__eq__' of Bundle object at 0x722ee9558b00>),
 ('__format__', <function Bundle.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of Bundle object at 0x722ee9558b00>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of Bundle object at 0x722ee9558b00>),
 ('__gt__', <method-wrapper '__gt__' of Bundle object at 0x722ee9558b00>),
 ('__hash__', <method-wrapper '__hash__' of Bundle object at 0x722ee9558b00>),
 ('__init__',
  <bound method Bundl

In [10]:


talb_doc[1].get_tree().descendants[1]

Node<sv-ud-train-2#2, skattereformen>

In [53]:
bundle = talb_doc[0]
root = bundle.get_tree()
nodes = root.descendants



node = nodes[0]

if node._parent is None:
    head = '_' # Empty nodes
else:
    try:
        head = str(node._parent._ord)
    except AttributeError:
        head = '0'

line = '\t'.join('_' if v is None else v for v in
                (str(node._ord), node.form, node.lemma, node.upos, node.xpos,
                '_' if node._feats is None else str(node.feats), head, node.deprel,
                node.raw_deps, '_' if node._misc is None else str(node.misc)))

len(line.split('\t'))


10

In [3]:
fixed_exp_df = pd.read_csv('/home/norrman/GitHub/UD_Swedish-Talbanken/not-to-release/fixed_exp.tsv', sep='\t')
fixed_exp_df.head()

,Expression,Transl.,Freq,Talb.,Lines,UPOS,Separable,Iterable,Idiom part,Morph status,Modif.,example,Infls,Synyms,Order,Abbr.,Pattern,Opt. part,Collapsible,Alts
0,som om,as if,73.0,0.0,73.0,SCONJ,No,No,No,Roots,No,No,No,No,No,No,som SC,No,No,NaN
1,så att,so that,70.0,34.0,36.0,SCONJ,No,No,No,Roots,No,No,No,No,No,No,ADV SC,Yes,No,NaN
2,det här,this,61.0,14.0,47.0,DET/PRON,No,No,No,Roots,No,No,Yes,No,No,No,DET ADV,No,No,NaN
3,även om,even if,43.0,31.0,12.0,SCONJ,No,No,No,1Infl,No,No,No,Yes,No,No,ADV SC,No,No,NaN
4,i dag,today,43.0,39.0,4.0,ADV,No,No,No,Roots,No,No,No,No,No,No,P NN,No,Yes,idag


In [4]:
def get_fixed_exp(treebanks):
    if not isinstance(treebanks, list):
        treebanks = [treebanks]
    fixed_exp = list()
    for treebank in treebanks:
        for node in treebank.nodes:
            if node.deprel == 'fixed' and node.parent.deprel != 'fixed':
                if node.parent not in fixed_exp:
                    fixed_exp.append(node.parent)
    fixed_exp = [sorted([exp, *[child for child in exp.children if child.deprel == 'fixed']], key=lambda node: node.ord) for exp in fixed_exp]
    fixed_exp_text = [{'expression': ' '.join([node.form for node in subtree]).lower(), 'text': subtree[0]._root.compute_text()} for subtree in fixed_exp]  
    
    occurences = collections.Counter([item['expression'] for item in fixed_exp_text])

    return occurences, fixed_exp_text


In [5]:
talb_occ, talb_fixed = get_fixed_exp(talb_doc)
pud_occ, pud_fixed = get_fixed_exp(pud_doc)
lines_occ, lines_fixed = get_fixed_exp([lines_train_doc, lines_dev_doc, lines_test_doc])

pud_occ

Counter({'till exempel': 2,
         'i jämförelse': 1,
         'den här': 15,
         'mer än': 7,
         'helt och hållet': 2,
         'i år': 2,
         'det här': 4,
         'i tisdags': 1,
         'vad för': 1,
         'i morse': 2,
         'till synes': 1,
         'för att': 2,
         'till sist': 4,
         'för närvarande': 1,
         'trots allt': 2,
         'så småningom': 5,
         'fler än': 1,
         'på grund av': 12,
         'som om': 1,
         'fram och tillbaka': 1,
         'så kallade': 2,
         'till slut': 2,
         'vad gäller': 2,
         'allt eftersom': 3,
         'så här': 1,
         'vid sidan av': 1,
         'med hjälp av': 1,
         'till och med': 2,
         'mer än bara': 1,
         'bak och fram': 1,
         'all dess': 1,
         'till följd av': 2,
         'i och med': 1,
         'i form av': 1,
         'i stället för': 1,
         'upp till': 2,
         'så mycket som': 1,
         'i allmänhet': 1,
         '

In [170]:
for exp, num in lines_occ.items():
    if fixed_exp_df.loc[fixed_exp_df['Expression'] == exp, 'Lines'].iloc[0] != num:
        print(exp, num, fixed_exp_df.loc[fixed_exp_df['Expression'] == exp, 'Lines'].iloc[0])

in i 27 26.0


In [142]:
all_fixed = set(list(lines_occ.keys()) + list(talb_occ.keys()) + list(pud_occ.keys()))
all_fixed

{'a priori',
 'a. a.',
 'ad calendas graecas',
 'aldrig så',
 'all dess',
 'allt annat än',
 'allt efter',
 'allt efter som',
 'allt eftersom',
 'allt fortfarande',
 'allt mer',
 'allt mera',
 'alltefter som',
 'annat än',
 'ansikte mot ansikte',
 'av allt att döma',
 'av principerna',
 'bak och fram',
 'beroende på',
 'bland andra',
 'bland annat',
 'bortsett från',
 'dag och natt',
 'de där',
 'de flesta',
 'de här',
 'den andra',
 'den där',
 'den här',
 'dess utveckling',
 'det där',
 'det här',
 'det samma',
 'det vill säga',
 'detta till trots',
 'dom där',
 'där inne',
 'därför att',
 'då det gäller',
 'då och då',
 'efter att',
 'efter det att',
 'efter hand',
 'efter hand som',
 'efter ämnet',
 'eller hur',
 'en del',
 'en eller annan',
 'en grupp',
 'en halv',
 'en hel del',
 'en i taget',
 'en känsla',
 'en massa',
 'en och',
 'en och annan',
 'en och en halv',
 'en och samma',
 'en viss',
 'ett annat',
 'ett dugg',
 'ett eller annat',
 'ett halvt',
 'ett tag',
 'fil lic',
 

In [158]:
fixed_exp_df.columns

Index(['Expression', 'Transl.', 'Freq', 'Talb.', 'Lines', 'UPOS', 'Separable',
       'Iterable', 'Idiom part', 'Morph status', 'Modif.', 'example', 'Infls',
       'Synyms', 'Order', 'Abbr.', 'Pattern', 'Opt. part', 'Collapsible',
       'Alts'],
      dtype='object')

In [164]:
occ_dict = {}
for exp in all_fixed:
    talb_count = talb_occ.get(exp, 0)
    lines_count = lines_occ.get(exp, 0)
    pud_count = pud_occ.get(exp, 0)
    freq = sum((talb_count, lines_count, pud_count))

    old_df_row = fixed_exp_df.loc[fixed_exp_df['Expression'] == exp] if exp in fixed_exp_df['Expression'].values else None

    occ_dict[exp] = {}
    occ_dict[exp]['Translation'] = old_df_row['Transl.'].iloc[0] if old_df_row is not None else None

    occ_dict[exp]['Freq'] = freq
    occ_dict[exp]['Talbanken'] = talb_count
    occ_dict[exp]['Lines'] = lines_count
    occ_dict[exp]['Pud'] = pud_count
    
    occ_dict[exp]['UPOS'] = old_df_row['UPOS'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Separable'] = old_df_row['Separable'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Iterable'] = old_df_row['Iterable'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Idiom part'] = old_df_row['Idiom part'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Morph status'] = old_df_row['Morph status'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Modif.'] = old_df_row['Modif.'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Example'] = old_df_row['example'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Infls'] = old_df_row['Infls'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Synyms'] = old_df_row['Synyms'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Abbr.'] = old_df_row['Abbr.'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Infls'] = old_df_row['Infls'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Pattern'] = old_df_row['Pattern'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Opt. part'] = old_df_row['Opt. part'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Collapsible'] = old_df_row['Collapsible'].iloc[0] if old_df_row is not None else None
    occ_dict[exp]['Alts'] = old_df_row['Alts'].iloc[0] if old_df_row is not None else None

    
# Create DataFrame
df = pd.DataFrame.from_dict(occ_dict, orient='index')

# Reset index to move the 'exp' values into a column
df.reset_index(inplace=True)
df.rename(columns={'index': 'expression'}, inplace=True)

df

,expression,Translation,Freq,Talbanken,Lines,Pud,UPOS,Separable,Iterable,Idiom part,Morph status,Modif.,Example,Infls,Synyms,Abbr.,Pattern,Opt. part,Collapsible,Alts
0,en och samma,one and the same,1,1,0,0,DET,No,No,No,Roots,No,No,Yes,No,No,DET CC JJ,No,No,NaN
1,av allt att döma,to all appearances,1,1,0,0,ADV,No,No,No,2Infl,No,No,No,Yes,No,P PR SC VB,No,No,NaN
2,rent av,downright,9,4,5,0,ADV,No,No,No,1Infl,No,No,No,No,No,X P,No,Yes,rentav
3,det där,that,20,1,19,0,DET/PRON,No,No,No,1Infl,No,No,Yes,No,No,DET ADV,No,No,NaN
4,i pedagogik,in pedagogy,1,1,0,0,ADV,Opt,No,No,Roots,Yes,klassisk,No,No,No,P NN,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,på rygg,on one’s back,3,3,0,0,ADV,Opt,No,No,Roots,Yes,sin,Yes,No,No,P NN,No,No,NaN
444,till låns,as a loan,1,1,0,0,ADV,No,No,No,2Spec,No,No,No,Yes,No,P NN,No,No,NaN
445,i sin tur,in his/her/its turn,9,8,0,1,ADV,No,No,No,Roots,No,No,Yes,No,No,P PS NN,No,No,NaN
446,till hands,at one’s disposal,2,2,0,0,ADV,No,No,No,2Spec,Yes,nära,No,No,No,P NN,No,Yes,tillhands


In [169]:
df.to_csv('fixed_expressions.tsv', sep='\t')

In [179]:
df.loc[(df['Talbanken']==0) & (df['Lines']==0) & (df['Pud']!=0), 'expression']

85     i mångt och mycket
91              efter att
133          i jämförelse
134             till dess
193           i linje med
219              all dess
277           mer än bara
352            så kallade
358          bak och fram
375             i tisdags
Name: expression, dtype: object

In [6]:
talb_m_train_doc = udapi.Document('/home/norrman/Downloads/sv_talbanken-ud-train.conllu')
talb_m_dev_doc = udapi.Document('/home/norrman/Downloads/sv_talbanken-ud-dev.conllu')
talb_m_test_doc = udapi.Document('/home/norrman/Downloads/sv_talbanken-ud-test.conllu')

talb_m_occ, talb_m_fixed = get_fixed_exp([talb_m_dev_doc, talb_m_test_doc, talb_m_train_doc])

In [9]:
for key, value in talb_m_occ.items():
    if talb_occ[key] != value:
        print(key, value, talb_occ[key])

sist berörda 1 0


In [8]:
# doc.store_conllu(filename='/home/norrman/GitHub/UD_Swedish-Talbanken/not-to-release/output/temp/udapi_test.conllu')